In [1]:
from __future__ import division 
from pyomo.environ import * 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = AbstractModel('TFG_MMP_v1.0_February_2023')

In [3]:
m.G = Set()
m.p = Set()
m.s = Set() 
m.n = Set()
m.t = Set()
m.h = Set()
m.hf = Set()

In [4]:
data = DataPortal()
data.load(filename= 'g.csv', set= m.G)
data.load(filename= 'p.csv', set= m.p)
data.load(filename= 's.csv', set= m.s)
data.load(filename= 'n.csv', set= m.n)
data.load(filename= 't.csv', set= m.t)
data.load(filename= 'h.csv', set= m.h)
data.load(filename= 'hf.csv', set= m.hf)

In [5]:
m.e = 480

In [6]:

m.CO2rate = Param(m.G, within = NonNegativeReals)
m.alfa = Param(m.G, within = NonNegativeReals)
m.beta = Param(m.G, within = NonNegativeReals)
m.gamma = Param(m.G, within = NonNegativeReals)
m.f = Param(m.G, within = NonNegativeReals)
m.o = Param(m.G, within = NonNegativeReals)
m.qmax = Param(m.G, within = NonNegativeReals)
m.qmin = Param(m.G, within = NonNegativeReals)
m.bmax = Param(m.G, within = NonNegativeReals)
m.wmax = Param(m.G)
m.w0 = Param(m.G)
m.wmin = Param(m.G)
m.k = Param(m.G)
m.rend = Param(m.G)

# Read CSV file into pandas DataFrame and convert to Pyomo parameter
param_data = pd.read_csv('Data_duration_demand1.csv', header=0, index_col=[0, 1, 2])
m.a = Param(m.n, m.s, m.p, initialize=param_data['a'].to_dict())
m.d = Param(m.n, m.s, m.p, initialize=param_data['d'].to_dict())

param_data2 = pd.read_csv('Data_inflows1.csv', header=0, index_col=[0, 1])
m.i = Param(m.h, m.p, initialize=param_data2['i'].to_dict())


param_data3 = pd.read_csv('Data_qred1.csv', header=0, index_col=[0, 1, 2, 3])
m.qred = Param(m.t, m.n, m.s, m.p, initialize=param_data3['qred'].to_dict())

data.load(filename="Data_generators1.csv", param=(m.CO2rate, m.alfa , m.beta , m.gamma , m.f , m.o , m.qmax , m.qmin , m.bmax , m.wmax , m.w0 , m.wmin , m.k , m.rend))
#instance = m.create_instance(data)
#instance.pprint()

In [7]:
m.u = Var(m.t, m.n, m.s, m.p,within= Binary) #u(t,s,p) Decision to connect generator g
m.y = Var(m.t,m.s, m.p, within= Binary)#y(t,s,p) Start up decision for generator g
#m.z = Var(m.G, within= Binary)#z(p,s,g) Shut down decision for generator g #IT IS NOT USED IN THE MODEL

In [8]:
m.qt = Var(m.t, m.n, m.s, m.p) #q(t,n,s,p) Net power delivered by thermal generator t [GW]
m.qh = Var(m.h, m.n, m.s, m.p) #q(h,n,s,p) Net power delivered by hydraulic generator h [GW]
m.b = Var(m.h, m.n, m.s, m.p) #b(h,n,s,p)Gross power consumed by pumping-storage plant [GW]
m.w = Var(m.h, m.p) #w(h,p) Hydroelectric energy in the reservoir of the plant [GWh]

In [9]:
def obj_rule (m):
    return sum((m.f[t]*(m.gamma[t]*m.y[t,s,p] + sum((m.a[n, s, p] * (m.alfa[t] * m.qt[t,n,s,p] / m.k [t] + m.beta[t] * m.u[t, n, s, p])) for n in m.n))  + 
        sum((m.a [n, s, p]* m.o[t] * m.qt[t, n, s, p] / m.k[t]) for n in m.n )) for t in m.t for p in m.p for s in m.s)

m.obj = Objective(rule= obj_rule, sense= minimize)

In [10]:
#DECLARATION OF CONSTRAINTS (meaning of warnings????)
    #Demand balance and marginal cost
def Demand_Bal_rule(m,n,s,p):
    return sum((m.qt[t,n,s,p]) for t in m.t) + sum((m.qh[h,n,s,p] - m.b[h,n,s,p]) for h in m.h) == m.d[n,s,p]

m.Demand_Bal = Constraint(m.n, m.s, m.p, rule= Demand_Bal_rule)

    #Upper thermal limit 
def Upper_Thermal_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] <= m.u[t,n,s,p] * m.k[t] * m.qmax[t]

m.Upper_Thermal = Constraint(m.t, m.n, m.s, m.p, rule= Upper_Thermal_rule)
    #Upper hydro constraints
def Upper_Hydro_rule(m,h,n,s,p):
    return m.qh[h,n,s,p] <= m.k[h] * m.qmax[h]

m.Upper_Hydro = Constraint(m.h, m.n, m.s, m.p, rule= Upper_Hydro_rule)
    #Maximum Capacity limits for pumped storage
def Cap_Hydro_rule(m,h,n,s,p):
    return m.b[h,n,s,p] <= m.k[h] * m.bmax[h]

m.Cap_Hydro = Constraint(m.h, m.n, m.s, m.p, rule= Cap_Hydro_rule)
    #Lower thermal limit
def Lower_Thermal_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] >= m.u[t,n,s,p] * m.k[t] * m.qmin[t]

m.Lower_Thermal = Constraint(m.t, m.n, m.s, m.p, rule= Lower_Thermal_rule)
    #Energy balance in an equivalent reservoir
def E_Bal_rule(m,h,p):
    if p == m.p.first():
        return Constraint.Skip
    else:
        return m.w[h,p] + sum(m.a[n,s,p] * (m.qh[h,n,s,p] - m.rend[h] * m.b[h,n,s,p]) for n in m.n for s in m.s) <= m.w[h,int(p) - 1] + m.i[h,p]

m.E_Bal = Constraint(m.h, m.p, rule= E_Bal_rule)
    #Upper limit equivalent energy stored in reservoir
def Upper_W_rule(m,h,p):
    return m.w[h,p] <= m.wmax[h]

m.Upper_W = Constraint(m.h, m.p, rule = Upper_W_rule)
    #Lower limit equivalent energy stored in basin
def Lower_W_rule(m,h,p):
    return m.w[h,p] >= m.wmin[h]

m.Lower_W = Constraint(m.h, m.p, rule = Lower_W_rule)
    #Minimum operating hours during the year
def Min_H_rule(m,t):
    return sum((m.a[n,s,p] * m.qt[t,n,s,p]) for n in m.n for s in m.s for p in m.p) >= m.k[t] * m.qmax[t] * m.e

m.Min_H = Constraint(m.t, rule = Min_H_rule)
    #Minimum production to accommodate grid constraints
def Min_Grid_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] >= m.qred[t,n,s,p]

m.Min_Grid = Constraint(m.t, m.n, m.s, m.p, rule = Min_Grid_rule)

In [11]:
workPath = os.getcwd()

In [12]:
print(data['G'],'\n')
print(data['p'],'\n')
print(data['s'],'\n')
print(data['n'],'\n')
print(data['t'],'\n')
print(data['h'],'\n')
print(data['hf'],'\n')
print(type(data['p'][10]))

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008', 'HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 

['lab', 'fes'] 

['n1', 'n2', 'n3', 'n4', 'n5'] 

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008'] 

['HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

['HYDRO_PUM'] 

<class 'int'>


In [13]:
instance = m.create_instance(data)
instance.pprint()

26 Set Declarations
    Cap_Hydro_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain  : Size : Members
        None :     4 : h*n*s*p :  360 : {('HYDRO_RES', 'n1', 'lab', 1), ('HYDRO_RES', 'n1', 'lab', 2), ('HYDRO_RES', 'n1', 'lab', 3), ('HYDRO_RES', 'n1', 'lab', 4), ('HYDRO_RES', 'n1', 'lab', 5), ('HYDRO_RES', 'n1', 'lab', 6), ('HYDRO_RES', 'n1', 'lab', 7), ('HYDRO_RES', 'n1', 'lab', 8), ('HYDRO_RES', 'n1', 'lab', 9), ('HYDRO_RES', 'n1', 'lab', 10), ('HYDRO_RES', 'n1', 'lab', 11), ('HYDRO_RES', 'n1', 'lab', 12), ('HYDRO_RES', 'n1', 'fes', 1), ('HYDRO_RES', 'n1', 'fes', 2), ('HYDRO_RES', 'n1', 'fes', 3), ('HYDRO_RES', 'n1', 'fes', 4), ('HYDRO_RES', 'n1', 'fes', 5), ('HYDRO_RES', 'n1', 'fes', 6), ('HYDRO_RES', 'n1', 'fes', 7), ('HYDRO_RES', 'n1', 'fes', 8), ('HYDRO_RES', 'n1', 'fes', 9), ('HYDRO_RES', 'n1', 'fes', 10), ('HYDRO_RES', 'n1', 'fes', 11), ('HYDRO_RES', 'n1', 'fes', 12), ('HYDRO_RES', 'n2', 'lab', 1), ('HYDRO_RES', 'n2', 'lab', 2), ('HYDRO_RES', 'n2', 'lab

        ('n3', 'lab', 11) :   200
        ('n3', 'lab', 12) :   210
         ('n4', 'fes', 1) :    23
         ('n4', 'fes', 2) :    24
         ('n4', 'fes', 3) :    49
         ('n4', 'fes', 4) :    26
         ('n4', 'fes', 5) :    27
         ('n4', 'fes', 6) :    28
         ('n4', 'fes', 7) :    29
         ('n4', 'fes', 8) :    30
         ('n4', 'fes', 9) :    31
        ('n4', 'fes', 10) :    32
        ('n4', 'fes', 11) :    33
        ('n4', 'fes', 12) :    34
         ('n4', 'lab', 1) :   148
         ('n4', 'lab', 2) :   139
         ('n4', 'lab', 3) :    84
         ('n4', 'lab', 4) :   137
         ('n4', 'lab', 5) :   144
         ('n4', 'lab', 6) :   143
         ('n4', 'lab', 7) :   158
         ('n4', 'lab', 8) :   133
         ('n4', 'lab', 9) :   148
        ('n4', 'lab', 10) :   155
        ('n4', 'lab', 11) :   130
        ('n4', 'lab', 12) :   137
         ('n5', 'fes', 1) :     0
         ('n5', 'fes', 2) :     0
         ('n5', 'fes', 3) :     0
         ('n5'

         ('GEN_004', 'n3', 'lab', 8) :   0.0
         ('GEN_004', 'n3', 'lab', 9) :   0.0
        ('GEN_004', 'n3', 'lab', 10) :   0.0
        ('GEN_004', 'n3', 'lab', 11) :   0.0
        ('GEN_004', 'n3', 'lab', 12) :   0.0
         ('GEN_004', 'n4', 'fes', 1) :   0.0
         ('GEN_004', 'n4', 'fes', 2) :   0.0
         ('GEN_004', 'n4', 'fes', 3) :   0.0
         ('GEN_004', 'n4', 'fes', 4) :   0.0
         ('GEN_004', 'n4', 'fes', 5) :   0.0
         ('GEN_004', 'n4', 'fes', 6) :   0.0
         ('GEN_004', 'n4', 'fes', 7) :   0.0
         ('GEN_004', 'n4', 'fes', 8) :   0.0
         ('GEN_004', 'n4', 'fes', 9) :   0.0
        ('GEN_004', 'n4', 'fes', 10) :   0.0
        ('GEN_004', 'n4', 'fes', 11) :   0.0
        ('GEN_004', 'n4', 'fes', 12) :   0.0
         ('GEN_004', 'n4', 'lab', 1) :   0.0
         ('GEN_004', 'n4', 'lab', 2) :   0.0
         ('GEN_004', 'n4', 'lab', 3) :   0.0
         ('GEN_004', 'n4', 'lab', 4) :   0.0
         ('GEN_004', 'n4', 'lab', 5) :   0.0
         (

         ('GEN_007', 'n3', 'lab', 7) :   0.0
         ('GEN_007', 'n3', 'lab', 8) :   0.0
         ('GEN_007', 'n3', 'lab', 9) :   0.0
        ('GEN_007', 'n3', 'lab', 10) :   0.0
        ('GEN_007', 'n3', 'lab', 11) :   0.0
        ('GEN_007', 'n3', 'lab', 12) :   0.0
         ('GEN_007', 'n4', 'fes', 1) :   0.0
         ('GEN_007', 'n4', 'fes', 2) :   0.0
         ('GEN_007', 'n4', 'fes', 3) :   0.0
         ('GEN_007', 'n4', 'fes', 4) :   0.0
         ('GEN_007', 'n4', 'fes', 5) :   0.0
         ('GEN_007', 'n4', 'fes', 6) :   0.0
         ('GEN_007', 'n4', 'fes', 7) :   0.0
         ('GEN_007', 'n4', 'fes', 8) :   0.0
         ('GEN_007', 'n4', 'fes', 9) :   0.0
        ('GEN_007', 'n4', 'fes', 10) :   0.0
        ('GEN_007', 'n4', 'fes', 11) :   0.0
        ('GEN_007', 'n4', 'fes', 12) :   0.0
         ('GEN_007', 'n4', 'lab', 1) :   0.0
         ('GEN_007', 'n4', 'lab', 2) :   0.0
         ('GEN_007', 'n4', 'lab', 3) :   0.0
         ('GEN_007', 'n4', 'lab', 4) :   0.0
         (

          GEN_006 :     0
          GEN_007 :     0
          GEN_008 :     0
        HYDRO_PUM :     0
        HYDRO_RES :  1200
        HYDRO_ROR :     0
    wmax : Size=11, Index=G, Domain=Any, Default=None, Mutable=False
        Key       : Value
          GEN_001 :     0
          GEN_002 :     0
          GEN_003 :     0
          GEN_004 :     0
          GEN_005 :     0
          GEN_006 :     0
          GEN_007 :     0
          GEN_008 :     0
        HYDRO_PUM :     0
        HYDRO_RES :  1400
        HYDRO_ROR :     0
    wmin : Size=11, Index=G, Domain=Any, Default=None, Mutable=False
        Key       : Value
          GEN_001 :     0
          GEN_002 :     0
          GEN_003 :     0
          GEN_004 :     0
          GEN_005 :     0
          GEN_006 :     0
          GEN_007 :     0
          GEN_008 :     0
        HYDRO_PUM :     0
        HYDRO_RES :  1000
        HYDRO_ROR :     0

6 Var Declarations
    b : Size=360, Index=b_index
        Key                   

...

        Key     : Lower              : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

         ('HYDRO_ROR', 'n5', 'lab', 8) :  -Inf :  qh[HYDRO_ROR,n5,lab,8] :   0.5 :   True
         ('HYDRO_ROR', 'n5', 'lab', 9) :  -Inf :  qh[HYDRO_ROR,n5,lab,9] :   0.5 :   True
        ('HYDRO_ROR', 'n5', 'lab', 10) :  -Inf : qh[HYDRO_ROR,n5,lab,10] :   0.5 :   True
        ('HYDRO_ROR', 'n5', 'lab', 11) :  -Inf : qh[HYDRO_ROR,n5,lab,11] :   0.5 :   True
        ('HYDRO_ROR', 'n5', 'lab', 12) :  -Inf : qh[HYDRO_ROR,n5,lab,12] :   0.5 :   True
    Upper_Thermal : Size=960, Index=Upper_Thermal_index, Active=True
        Key                          : Lower : Body                                                             : Upper : Active
         ('GEN_001', 'n1', 'fes', 1) :  -Inf :                  qt[GEN_001,n1,fes,1] - 0.95*u[GEN_001,n1,fes,1] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 2) :  -Inf :                  qt[GEN_001,n1,fes,2] - 0.95*u[GEN_001,n1,fes,2] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 3) :  -Inf :                  qt[GEN_001,n1,fes,3] - 0.95*u[

         ('GEN_003', 'n2', 'lab', 8) :  -Inf :                qt[GEN_003,n2,lab,8] - 0.3255*u[GEN_003,n2,lab,8] :   0.0 :   True
         ('GEN_003', 'n2', 'lab', 9) :  -Inf :                qt[GEN_003,n2,lab,9] - 0.3255*u[GEN_003,n2,lab,9] :   0.0 :   True
        ('GEN_003', 'n2', 'lab', 10) :  -Inf :              qt[GEN_003,n2,lab,10] - 0.3255*u[GEN_003,n2,lab,10] :   0.0 :   True
        ('GEN_003', 'n2', 'lab', 11) :  -Inf :              qt[GEN_003,n2,lab,11] - 0.3255*u[GEN_003,n2,lab,11] :   0.0 :   True
        ('GEN_003', 'n2', 'lab', 12) :  -Inf :              qt[GEN_003,n2,lab,12] - 0.3255*u[GEN_003,n2,lab,12] :   0.0 :   True
         ('GEN_003', 'n3', 'fes', 1) :  -Inf :                qt[GEN_003,n3,fes,1] - 0.3255*u[GEN_003,n3,fes,1] :   0.0 :   True
         ('GEN_003', 'n3', 'fes', 2) :  -Inf :                qt[GEN_003,n3,fes,2] - 0.3255*u[GEN_003,n3,fes,2] :   0.0 :   True
         ('GEN_003', 'n3', 'fes', 3) :  -Inf :                qt[GEN_003,n3,fes,3] - 0.3255*u[GEN

        ('GEN_005', 'n3', 'fes', 12) :  -Inf : qt[GEN_005,n3,fes,12] - 0.33249999999999996*u[GEN_005,n3,fes,12] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 1) :  -Inf :   qt[GEN_005,n3,lab,1] - 0.33249999999999996*u[GEN_005,n3,lab,1] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 2) :  -Inf :   qt[GEN_005,n3,lab,2] - 0.33249999999999996*u[GEN_005,n3,lab,2] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 3) :  -Inf :   qt[GEN_005,n3,lab,3] - 0.33249999999999996*u[GEN_005,n3,lab,3] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 4) :  -Inf :   qt[GEN_005,n3,lab,4] - 0.33249999999999996*u[GEN_005,n3,lab,4] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 5) :  -Inf :   qt[GEN_005,n3,lab,5] - 0.33249999999999996*u[GEN_005,n3,lab,5] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 6) :  -Inf :   qt[GEN_005,n3,lab,6] - 0.33249999999999996*u[GEN_005,n3,lab,6] :   0.0 :   True
         ('GEN_005', 'n3', 'lab', 7) :  -Inf :   qt[GEN_005,n3,lab,7] - 0.33249999999999996*u[GEN

         ('GEN_007', 'n3', 'fes', 7) :  -Inf :                qt[GEN_007,n3,fes,7] - 0.5076*u[GEN_007,n3,fes,7] :   0.0 :   True
         ('GEN_007', 'n3', 'fes', 8) :  -Inf :                qt[GEN_007,n3,fes,8] - 0.5076*u[GEN_007,n3,fes,8] :   0.0 :   True
         ('GEN_007', 'n3', 'fes', 9) :  -Inf :                qt[GEN_007,n3,fes,9] - 0.5076*u[GEN_007,n3,fes,9] :   0.0 :   True
        ('GEN_007', 'n3', 'fes', 10) :  -Inf :              qt[GEN_007,n3,fes,10] - 0.5076*u[GEN_007,n3,fes,10] :   0.0 :   True
        ('GEN_007', 'n3', 'fes', 11) :  -Inf :              qt[GEN_007,n3,fes,11] - 0.5076*u[GEN_007,n3,fes,11] :   0.0 :   True
        ('GEN_007', 'n3', 'fes', 12) :  -Inf :              qt[GEN_007,n3,fes,12] - 0.5076*u[GEN_007,n3,fes,12] :   0.0 :   True
         ('GEN_007', 'n3', 'lab', 1) :  -Inf :                qt[GEN_007,n3,lab,1] - 0.5076*u[GEN_007,n3,lab,1] :   0.0 :   True
         ('GEN_007', 'n3', 'lab', 2) :  -Inf :                qt[GEN_007,n3,lab,2] - 0.5076*u[GEN

In [14]:
solver = SolverFactory('gurobi')
results = solver.solve(instance)
instance.pprint()
results.write()

26 Set Declarations
    Cap_Hydro_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain  : Size : Members
        None :     4 : h*n*s*p :  360 : {('HYDRO_RES', 'n1', 'lab', 1), ('HYDRO_RES', 'n1', 'lab', 2), ('HYDRO_RES', 'n1', 'lab', 3), ('HYDRO_RES', 'n1', 'lab', 4), ('HYDRO_RES', 'n1', 'lab', 5), ('HYDRO_RES', 'n1', 'lab', 6), ('HYDRO_RES', 'n1', 'lab', 7), ('HYDRO_RES', 'n1', 'lab', 8), ('HYDRO_RES', 'n1', 'lab', 9), ('HYDRO_RES', 'n1', 'lab', 10), ('HYDRO_RES', 'n1', 'lab', 11), ('HYDRO_RES', 'n1', 'lab', 12), ('HYDRO_RES', 'n1', 'fes', 1), ('HYDRO_RES', 'n1', 'fes', 2), ('HYDRO_RES', 'n1', 'fes', 3), ('HYDRO_RES', 'n1', 'fes', 4), ('HYDRO_RES', 'n1', 'fes', 5), ('HYDRO_RES', 'n1', 'fes', 6), ('HYDRO_RES', 'n1', 'fes', 7), ('HYDRO_RES', 'n1', 'fes', 8), ('HYDRO_RES', 'n1', 'fes', 9), ('HYDRO_RES', 'n1', 'fes', 10), ('HYDRO_RES', 'n1', 'fes', 11), ('HYDRO_RES', 'n1', 'fes', 12), ('HYDRO_RES', 'n2', 'lab', 1), ('HYDRO_RES', 'n2', 'lab', 2), ('HYDRO_RES', 'n2', 'lab

    i : Size=36, Index=i_index, Domain=Any, Default=None, Mutable=False
        Key               : Value
         ('HYDRO_PUM', 1) :     0
         ('HYDRO_PUM', 2) :     0
         ('HYDRO_PUM', 3) :     0
         ('HYDRO_PUM', 4) :     0
         ('HYDRO_PUM', 5) :     0
         ('HYDRO_PUM', 6) :     0
         ('HYDRO_PUM', 7) :     0
         ('HYDRO_PUM', 8) :     0
         ('HYDRO_PUM', 9) :     0
        ('HYDRO_PUM', 10) :     0
        ('HYDRO_PUM', 11) :     0
        ('HYDRO_PUM', 12) :     0
         ('HYDRO_RES', 1) :   400
         ('HYDRO_RES', 2) :   320
         ('HYDRO_RES', 3) :   340
         ('HYDRO_RES', 4) :   260
         ('HYDRO_RES', 5) :   200
         ('HYDRO_RES', 6) :   150
         ('HYDRO_RES', 7) :   100
         ('HYDRO_RES', 8) :   100
         ('HYDRO_RES', 9) :   260
        ('HYDRO_RES', 10) :   280
        ('HYDRO_RES', 11) :   260
        ('HYDRO_RES', 12) :   320
         ('HYDRO_ROR', 1) :   300
         ('HYDRO_ROR', 2) :   225
         (

         ('GEN_004', 'n3', 'fes', 9) :   0.0
        ('GEN_004', 'n3', 'fes', 10) :   0.0
        ('GEN_004', 'n3', 'fes', 11) :   0.0
        ('GEN_004', 'n3', 'fes', 12) :   0.0
         ('GEN_004', 'n3', 'lab', 1) :   0.0
         ('GEN_004', 'n3', 'lab', 2) :   0.0
         ('GEN_004', 'n3', 'lab', 3) :   0.0
         ('GEN_004', 'n3', 'lab', 4) :   0.0
         ('GEN_004', 'n3', 'lab', 5) :   0.0
         ('GEN_004', 'n3', 'lab', 6) :   0.0
         ('GEN_004', 'n3', 'lab', 7) :   0.0
         ('GEN_004', 'n3', 'lab', 8) :   0.0
         ('GEN_004', 'n3', 'lab', 9) :   0.0
        ('GEN_004', 'n3', 'lab', 10) :   0.0
        ('GEN_004', 'n3', 'lab', 11) :   0.0
        ('GEN_004', 'n3', 'lab', 12) :   0.0
         ('GEN_004', 'n4', 'fes', 1) :   0.0
         ('GEN_004', 'n4', 'fes', 2) :   0.0
         ('GEN_004', 'n4', 'fes', 3) :   0.0
         ('GEN_004', 'n4', 'fes', 4) :   0.0
         ('GEN_004', 'n4', 'fes', 5) :   0.0
         ('GEN_004', 'n4', 'fes', 6) :   0.0
         (

         ('GEN_007', 'n4', 'lab', 9) :   0.0
        ('GEN_007', 'n4', 'lab', 10) :   0.0
        ('GEN_007', 'n4', 'lab', 11) :   0.0
        ('GEN_007', 'n4', 'lab', 12) :   0.0
         ('GEN_007', 'n5', 'fes', 1) :   0.0
         ('GEN_007', 'n5', 'fes', 2) :   0.0
         ('GEN_007', 'n5', 'fes', 3) :   0.0
         ('GEN_007', 'n5', 'fes', 4) :   0.0
         ('GEN_007', 'n5', 'fes', 5) :   0.0
         ('GEN_007', 'n5', 'fes', 6) :   0.0
         ('GEN_007', 'n5', 'fes', 7) :   0.0
         ('GEN_007', 'n5', 'fes', 8) :   0.0
         ('GEN_007', 'n5', 'fes', 9) :   0.0
        ('GEN_007', 'n5', 'fes', 10) :   0.0
        ('GEN_007', 'n5', 'fes', 11) :   0.0
        ('GEN_007', 'n5', 'fes', 12) :   0.0
         ('GEN_007', 'n5', 'lab', 1) :   0.0
         ('GEN_007', 'n5', 'lab', 2) :   0.0
         ('GEN_007', 'n5', 'lab', 3) :   0.0
         ('GEN_007', 'n5', 'lab', 4) :   0.0
         ('GEN_007', 'n5', 'lab', 5) :   0.0
         ('GEN_007', 'n5', 'lab', 6) :   0.0
         (

         ('HYDRO_RES', 'n4', 'lab', 2) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 3) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 4) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 5) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 6) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 7) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 8) :  None :                  0.1 :  None : False : False :  Reals
         ('HYDRO_RES', 'n4', 'lab', 9) :  None :                  0.1 :  None : False : False :  Reals
        ('HYDRO_RES', 'n4', 'lab', 10) :  None :                  0.1 :  None : False : False :  Reals
        ('HYDRO_RES', 'n4', 'lab', 11) :  None :                  0.1 :  

...

        ('GEN_001', 'n4', 'lab', 11) :   0.0 : qt[GEN_001,n4,lab,11] :  +Inf :   True
        ('GEN_001', 'n4', 'lab', 12) :   0.0 : qt[GEN_001,n4,lab,12] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 1) :   0.0 :  qt[GEN_001,n5,fes,1] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 2) :   0.0 :  qt[GEN_001,n5,fes,2] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 3) :   0.0 :  qt[GEN_001,n5,fes,3] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 4) :   0.0 :  qt[GEN_001,n5,fes,4] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 5) :   0.0 :  qt[GEN_001,n5,fes,5] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 6) :   0.0 :  qt[GEN_001,n5,fes,6] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 7) :   0.0 :  qt[GEN_001,n5,fes,7] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 8) :   0.0 :  qt[GEN_001,n5,fes,8] :  +Inf :   True
         ('GEN_001', 'n5', 'fes', 9) :   0.0 :  qt[GEN_001,n5,fes,9] :  +Inf :   True
        ('GEN_001', 'n5', 'fes', 10) :   0.0 : qt[GEN_

         ('GEN_006', 'n1', 'fes', 5) :   0.0 :  qt[GEN_006,n1,fes,5] :  +Inf :   True
         ('GEN_006', 'n1', 'fes', 6) :   0.0 :  qt[GEN_006,n1,fes,6] :  +Inf :   True
         ('GEN_006', 'n1', 'fes', 7) :   0.0 :  qt[GEN_006,n1,fes,7] :  +Inf :   True
         ('GEN_006', 'n1', 'fes', 8) :   0.0 :  qt[GEN_006,n1,fes,8] :  +Inf :   True
         ('GEN_006', 'n1', 'fes', 9) :   0.0 :  qt[GEN_006,n1,fes,9] :  +Inf :   True
        ('GEN_006', 'n1', 'fes', 10) :   0.0 : qt[GEN_006,n1,fes,10] :  +Inf :   True
        ('GEN_006', 'n1', 'fes', 11) :   0.0 : qt[GEN_006,n1,fes,11] :  +Inf :   True
        ('GEN_006', 'n1', 'fes', 12) :   0.0 : qt[GEN_006,n1,fes,12] :  +Inf :   True
         ('GEN_006', 'n1', 'lab', 1) :   0.0 :  qt[GEN_006,n1,lab,1] :  +Inf :   True
         ('GEN_006', 'n1', 'lab', 2) :   0.0 :  qt[GEN_006,n1,lab,2] :  +Inf :   True
         ('GEN_006', 'n1', 'lab', 3) :   0.0 :  qt[GEN_006,n1,lab,3] :  +Inf :   True
         ('GEN_006', 'n1', 'lab', 4) :   0.0 :  qt[GEN

        Key     : Lower              : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

        Key                          : Lower : Body                                                             : Upper : Active
         ('GEN_001', 'n1', 'fes', 1) :  -Inf :                  qt[GEN_001,n1,fes,1] - 0.95*u[GEN_001,n1,fes,1] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 2) :  -Inf :                  qt[GEN_001,n1,fes,2] - 0.95*u[GEN_001,n1,fes,2] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 3) :  -Inf :                  qt[GEN_001,n1,fes,3] - 0.95*u[GEN_001,n1,fes,3] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 4) :  -Inf :                  qt[GEN_001,n1,fes,4] - 0.95*u[GEN_001,n1,fes,4] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 5) :  -Inf :                  qt[GEN_001,n1,fes,5] - 0.95*u[GEN_001,n1,fes,5] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 6) :  -Inf :                  qt[GEN_001,n1,fes,6] - 0.95*u[GEN_001,n1,fes,6] :   0.0 :   True
         ('GEN_001', 'n1', 'fes', 7) :  -Inf :                  qt[GEN_001,n1,fes,7] - 0.95*u[GEN

         ('GEN_003', 'n4', 'fes', 8) :  -Inf :                qt[GEN_003,n4,fes,8] - 0.3255*u[GEN_003,n4,fes,8] :   0.0 :   True
         ('GEN_003', 'n4', 'fes', 9) :  -Inf :                qt[GEN_003,n4,fes,9] - 0.3255*u[GEN_003,n4,fes,9] :   0.0 :   True
        ('GEN_003', 'n4', 'fes', 10) :  -Inf :              qt[GEN_003,n4,fes,10] - 0.3255*u[GEN_003,n4,fes,10] :   0.0 :   True
        ('GEN_003', 'n4', 'fes', 11) :  -Inf :              qt[GEN_003,n4,fes,11] - 0.3255*u[GEN_003,n4,fes,11] :   0.0 :   True
        ('GEN_003', 'n4', 'fes', 12) :  -Inf :              qt[GEN_003,n4,fes,12] - 0.3255*u[GEN_003,n4,fes,12] :   0.0 :   True
         ('GEN_003', 'n4', 'lab', 1) :  -Inf :                qt[GEN_003,n4,lab,1] - 0.3255*u[GEN_003,n4,lab,1] :   0.0 :   True
         ('GEN_003', 'n4', 'lab', 2) :  -Inf :                qt[GEN_003,n4,lab,2] - 0.3255*u[GEN_003,n4,lab,2] :   0.0 :   True
         ('GEN_003', 'n4', 'lab', 3) :  -Inf :                qt[GEN_003,n4,lab,3] - 0.3255*u[GEN

In [19]:
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  result.solver.status)

feasible


In [ ]:
#print('u=',value(instance.u['GEN_001']['n1']['lab'][1]))
#print('y=',round(value(instance.y),2))
#print('w=',round(value(instance.w),2))